## Notes:
----------

Theory:
We have see a lot of discreapancy with the images, hence for testing puspose we only select images that were last_reviewed after September 2017. We believe that the data atleast reviewed after september should be correct.

1. The accessor photo column had some "nan" values, these were removed.
2. 




## WHAT THE BELOW CELLS DO:

1. We load the metadata file that contains the property metadata for appx 250000 properties including the assessor image.
3. We dont want to experiment on all the images, so we select the ones that were last_reviewed after April 2017. We further filter the images. For land the images that have a improvement level as land are only selected. For House, images with improvement level of anything other than land is taken.
4. We then collect all the assessor images from the links provided in the metadata table. Somtimes the images are not rendered properly, so we filter such images
5. Similarly we gather all the Aerial images using Bing map API call
6. Lastely we acquire all streeside images from Google.
7. We can even gather all the Aerial images using Google map API call

In [15]:
import os
import random
import pandas as pd
import numpy as np
import logging
from semantic_segmentation import utils as utl
from semantic_segmentation.overlay_building_bbox import overlay_parcel_on_images

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w", 
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

from external_data import google_aerial # bing_aerial, google_streetside, 
from config import pathDict

parent_path = pathDict['parent_path']
xlsx_file = os.path.join(parent_path, "Image_Classification_Project.xlsx")
meta_data_path = os.path.join(parent_path, "house_metadata_nw.csv")
assessor_image_path = os.path.join(parent_path, "input_images", "assessor_images")
# bing_aerial_image_path = os.path.join(parent_path, 'input_images','bing','aerial_images')
google_aerial_image_path = os.path.join(parent_path, 'input_images','google','aerial_images')
streetside_image_path = os.path.join(parent_path, 'input_images','streetside_images')

print('Metadata path: ', meta_data_path)
print('Assessor image path: ', assessor_image_path)
# print('Aerial image path (Bing): ', bing_aerial_image_path)
print('Streetside image path: ', streetside_image_path)
print('Aerial image path (Google): ', google_aerial_image_path)

xlsx_to_csv = False
dump_aerial_google_images  = True
overlay_bbox = True

Metadata path:  /Users/sam/All-Program/App-DataSet/HouseClassification/training_data/house_metadata_nw.csv
Assessor image path:  /Users/sam/All-Program/App-DataSet/HouseClassification/training_data/input_images/assessor_images
Streetside image path:  /Users/sam/All-Program/App-DataSet/HouseClassification/training_data/input_images/streetside_images
Aerial image path (Google):  /Users/sam/All-Program/App-DataSet/HouseClassification/training_data/input_images/google/aerial_images


### LOAD THE XLSX FILE GET DATA AND PUT INTO A NEW CSV FILE

In [2]:
if xlsx_to_csv:
    data_xls = pd.read_excel(xlsx_file, 'Data Set', index_col=None)
    data_xls.to_csv(meta_data_path, encoding='utf-8')

In [3]:
def collateData(filesPath, fileType="csv"):
    fileDirList = [os.path.join(filesPath, files) for files in
                   os.listdir(filesPath) if files.endswith('.%s' % fileType)]
    if fileType == "csv":
        rowList = []
        # fileDirList = [files for files in listdir if files.endswith('.csv')]
        for files in fileDirList:
            df = pd.read_csv(files, header=0)
            rowList.append(df)
        return pd.concat(rowList)
    elif fileType == "json":
        dataOUT = {}
        for files in fileDirList:
            jsonFilepath = os.path.join(filesPath, files)
            with open(jsonFilepath, 'r') as fileIN:
                dataOUT = merge(dataOUT, json.load(fileIN))
        return dataOUT

# STEP 1: Load Metadata file
------------

In [4]:
metadata = pd.read_csv(meta_data_path)
metadata.columns = ['row_id', 'removed', 'property_id', 'state', 'county_name', 'pin',
       'address_line1', 'address_line2', 'address_city', 'address_zip',
       'zoning', 'improvement_level', 'type', 'exterior',
       'last_reviewed_timestamp', 'gone_timestamp', 'indicator',
       'assessor_photo']
metadata['state'] = metadata['state'].astype('str')
metadata['county_name'] = metadata['county_name'].astype('str')
metadata['pin'] = metadata['pin'].astype('str')
metadata['address_line1'] = metadata['address_line1'].astype('str')
metadata['address_line2'] = metadata['address_line2'].astype('str')
metadata['address_city'] = metadata['address_city'].astype('str')
metadata['address_zip'] = metadata['address_zip'].astype('str')
metadata['zoning'] = metadata['zoning'].astype('str')
metadata['improvement_level'] = metadata['improvement_level'].astype('str')
metadata['type'] = metadata['type'].astype('str')
metadata['exterior'] = metadata['exterior'].astype('str')
metadata['last_reviewed_timestamp'] = pd.to_datetime(metadata['last_reviewed_timestamp'])#.astype('str')
metadata['gone_timestamp'] = pd.to_datetime(metadata['gone_timestamp'])#.astype('str')
metadata['indicator'] = metadata['indicator'].astype('str')
metadata['assessor_photo'] = metadata['assessor_photo'].astype('str')

# Remove the Test Dat from Metadata
metadata = metadata[metadata['removed'] == 0]
print(metadata.shape)
metadata.head(3)

/Users/sam/App-Setup/anaconda/envs/anaconda35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(220364, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
1,1,0,499849,IL,Cook,01-01-100-109-0000,200S HOUGH ST,nan,BARRINGTON,nan,Vacant,Land,nan,nan,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,2,0,499852,IL,Cook,01-01-101-029-0000,201 First,22 # Floor,BARRINGTON,nan,Vacant,Land,4 Unit Apt Bldg,Frame,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
3,3,0,499854,IL,Cook,01-01-109-001-0000,323 MAIN ST,nan,BARRINGTON,600104305.0,Residential,Improved,2 Flat,Frame,2017-05-25 04:42:26.850,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


# STEP 2: Create a new Subset
--------------

In [5]:
metadata_lst_rvd_valid = metadata[metadata['last_reviewed_timestamp'] != '']
metadata_lst_rvd_valid['last_reviewed_timestamp'] = pd.to_datetime(metadata_lst_rvd_valid['last_reviewed_timestamp'])
metadata_lst_rvd_valid_2017 = metadata_lst_rvd_valid[((metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.year == 2017) & 
                                                     (metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.month >=1 ) &
#                                                       (metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.month <7) &
                                                      (metadata_lst_rvd_valid['address_city'].str.lower().str.strip().str.match('chicago')) &
                                                      (metadata_lst_rvd_valid['address_line1'] != 'nan') &
                                                      (metadata_lst_rvd_valid['assessor_photo'] != 'nan') &
                                                     (metadata_lst_rvd_valid['indicator'].isin(["Likely House", "Likely Land"]) ))]
image_data = metadata_lst_rvd_valid_2017
print (metadata_lst_rvd_valid_2017.shape)
metadata_lst_rvd_valid_2017.head(2)

(28527, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### LAND DATA
----------------

In [6]:
land_data = image_data[(image_data['indicator'] == 'Likely Land') & (image_data['improvement_level'] == 'Land')]
print (np.unique(np.array(land_data['improvement_level'])))
print (land_data.shape)
land_data.head(3)

['Land']
(9873, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9339,9339,0,510240,IL,Cook,10-31-417-045-0000,6546 W DEVON AVE,nan,CHICAGO,606312038.0,Vacant,Land,nan,nan,2017-06-22 18:07:04.937,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9366,9366,0,510267,IL,Cook,10-32-126-007-0000,7046N MCALPIN AVE,nan,CHICAGO,60646.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### HOUSE DATA
--------------

In [7]:
house_data = image_data[(image_data['indicator'] == 'Likely House') & (image_data['improvement_level'] != 'Land')]
print (np.unique(np.array(house_data['improvement_level'])))
print (house_data.shape)
house_data.head(3)

['Condominium' 'Improved' 'Minor Improvement' 'Mixed Use']
(18392, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8761,8761,0,509531,IL,Cook,10-25-308-043-0000,3146W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-10 21:21:30.520,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8769,8769,0,509539,IL,Cook,10-25-314-003-0000,2949W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-23 16:25:51.033,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### UNKNOWN DATA
-------------

In [8]:
unknown_data = image_data[image_data['indicator'] == 'Unknown']
print (unknown_data.shape)
unknown_data.head(3)

(0, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo


# Step 1: Load Aerial Images (Google Static Maps):
------------

In [31]:
which_run='1522630301'
from sklearn.utils import shuffle
import shutil
import time

class GetAerial():
    def __init__(self, zoom, state, map_size, batch_size, which_run):
        self.zoom = zoom
        self.state = state
        self.map_size = map_size
        self.batch_size = batch_size
        self.which_run = which_run
        
    def parse_dump_aerial(self, land_house_metadata):
        if self.which_run != 'latest':
            stats_path = os.path.join(pathDict['aerial_stats_path'], self.which_run)
            if os.path.exists(stats_path):
                if len(os.listdir(stats_path)) > 0:
                    aerial_stats = collateData(filesPath=stats_path, fileType="csv")
                    processed_images = aerial_stats.dropna()
                    processed_images = processed_images[processed_images['meta_url'] != 'EXCEED']
                    print ('Number of images already parsed and dumped ', len(aerial_stats))
                    
                    # Delete the seperate batch files batch files
                    shutil.rmtree(stats_path + '/')
                    os.makedirs(stats_path)
                    
                    # Combine the batchfiles into 1 and save
                    processed_images = processed_images[['pin', 'address', 'city', 'lat', 'lon','loc_type', 'indicator', 'meta_url', 'img_url']]
                    processed_images.to_csv(os.path.join(stats_path, '%s_%s_%s.csv'%(str(time.time()).split('.')[0], str(0), str(len(processed_images)-1))), index=None)
                    land_house_metadata = land_house_metadata[~land_house_metadata['pin'].isin(np.array(processed_images['pin'], dtype=str))]
        
        land_house_metadata = shuffle(land_house_metadata)
        print ('Number of new images to be parsed and dumped ', len(land_house_metadata))
        google_aerial.fetch_google_aerial_images(land_house_metadata, zoom=self.zoom, state=self.state, map_size = self.map_size, batch_size=self.batch_size, get_stats=True, which_run=self.which_run)

### Remove from the land_house_data where the PIN is already processed

In [33]:
if dump_aerial_google_images:
    land_house_data = pd.concat([land_data, house_data])
    print (land_house_data.shape)
    pd.concat([land_house_data.head(2), land_house_data.tail(2)])
    GetAerial(zoom=20, state='IL', map_size = '400x400', batch_size=100, which_run='1522630301').parse_dump_aerial(land_house_data)

# Step 7: Here we randomly select a subset of the images, the conditions for selection are:

* Fisrst we run "get_valid_land_house_ids" under the main.py module to get images that were properly fetched for all different types of property images i.e "assessor", 'streetside', 'bing_aerial'.
* Now that we have Google Aerial Images metadata, we perform the below operation to do some supposedly "bad data" removal
    * A image should have a valid address_line1 and city. The address should not start with '0 0 GROVE AVE', '0 SUMMIT ST', '0 VACANT PROPERTY'
    * The location_type should not be "RANGE_INTERPOLATED"
    * The Lat and Lon should have a 4 or more than 4 decimal precision.
    

In [12]:
from main import get_valid_land_house_ids

valid_land_pins, valid_house_pins = get_valid_land_house_ids(images_per_label=None)
print ('Total qualifies valid pins each Label: ', len(valid_land_pins), len(valid_house_pins))

Total qualifies valid pins each Label:  6670 6670


In [13]:
google_stats_data = collateData(pathDict['google_aerial_stats_path'])
print (google_stats_data.shape)
google_stats_data.head()

(30364, 9)


,pin,address,city,lat,lon,loc_type,indicator,meta_url,img_url
0,20-28-324-003-0000,7809 S LOWE AVE,CHICAGO,41.752219,-87.640134,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
1,20-20-411-019-0000,6847 S CARPENTER ST,CHICAGO,41.769213,-87.650386,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
2,17-19-324-019-0000,2115 W 21ST PL,CHICAGO,41.852836,-87.679003,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
3,14-31-111-038-1001,2319 W LYNDALE ST,CHICAGO,41.922418,-87.685920,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
4,20-11-305-017-1001,5115 S UNIVERSITY AVE,CHICAGO,41.801807,-87.597783,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...


# Step 8: Overlay the building bounding box on Google satellite images:

In [16]:
which_run='1522630301'
if overlay_bbox:
    aerial_stats_path = os.path.join(pathDict['aerial_stats_path'], which_run)
    if not os.path.exists(aerial_stats_path):
        raise ValueError('The path %s seems to not exist, Make sure you have dumped the Aerial Images.', )
    aerial_stats_data = utl.collateData(aerial_stats_path)
    print('Shape: Google Stats images ', aerial_stats_data.shape)

    data_to_model = aerial_stats_data[(aerial_stats_data['loc_type'] != 'RANGE_INTERPOLATED') & (
    aerial_stats_data['city'].str.lower().str.strip().str.match('chicago')) & (~aerial_stats_data[
        'address'].str.lower().str.strip().str.match('0'))]
    print('Shape: Images for Overlaying: ', data_to_model.shape)
    overlay_parcel_on_images(data_to_model, zoom=20, map_size=[400, 400], which_run=which_run)

Shape: Google Stats images  (600, 9)
Shape: Images for Overlaying:  (580, 9)
Initiating Run for scoop scoop_738_768 : DATA SHAPE (93246, 13)

Initiating Run for scoop scoop_768_798 : DATA SHAPE (94331, 13)
TOTAL IMAGES PARSED ======== 99

Initiating Run for scoop scoop_798_848 : DATA SHAPE (80697, 13)

Initiating Run for scoop scoop_848_908 : DATA SHAPE (90177, 13)
TOTAL IMAGES PARSED ======== 99

Initiating Run for scoop scoop_908_938 : DATA SHAPE (92080, 13)

Initiating Run for scoop scoop_938_968 : DATA SHAPE (91240, 13)

Initiating Run for scoop scoop_968_028 : DATA SHAPE (86448, 13)



# Step 9: Select a subset of images for train and test